<a href="https://colab.research.google.com/github/GaborVxxx/ml_notes/blob/main/TextSummarizationInPythonEasyWay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:

# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

import pandas as pd
df = pd.read_csv('bbc_text_cls.csv')
df.head()

File ‘bbc_text_cls.csv’ already there; not retrieving.



,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [29]:
doc = df[df.labels == 'business']['text'].sample(random_state=42)

In [49]:
!pip install sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.4 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=8a565d40c94ce1ab483877518833a339e1bf803b308298d882416994b22acdb9
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=0fe6449f45bb71ddadded947a87c0b191b4a5b6a62bcf24b80dd5f95012834fc
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [50]:
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

In [52]:
summarizer = TextRankSummarizer()
parser = PlaintextParser.from_string(doc.iloc[0].split('\n', 1)[1], Tokenizer('english'))
summary = summarizer(parser.document, sentences_count=5)

In [53]:
summary

(<Sentence: Retail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said.>,
 <Sentence: The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%.>,
 <Sentence: The ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures.>,
 <Sentence: Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003.>,
 <Sentence: The November-December jump last year was roughly comparable with recent averages, although some way below the serious booms seen in the 1990s.>)

In [55]:
def wrap(x):
  return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [56]:
for s in summary:
  print(wrap(str(s)))

Retail sales dropped by 1% on the month in December, after a 0.6% rise
in November, the Office for National Statistics (ONS) said.
The ONS revised the annual 2004 rate of growth down from the 5.9%
estimated in November to 3.2%.
The ONS echoed an earlier caution from Bank of England governor Mervyn
King not to read too much into the poor December figures.
Some analysts put a positive gloss on the figures, pointing out that
the non-seasonally-adjusted figures showed a performance comparable
with 2003.
The November-December jump last year was roughly comparable with
recent averages, although some way below the serious booms seen in the
1990s.


In [57]:
summarizer = LsaSummarizer()
summary = summarizer(parser.document, sentences_count=5)
for s in summary:
  print(wrap(str(s)))

UK retail sales fell in December, failing to meet expectations and
making it by some counts the worst Christmas since 1981.
Morrisons, Woolworths, House of Fraser, Marks & Spencer and Big Food
all said that the festive period was disappointing.
And a British Retail Consortium survey found that Christmas 2004 was
the worst for 10 years.
Yet, other retailers - including HMV, Monsoon, Jessops, Body Shop and
Tesco - reported that festive sales were well up on last year.
Investec chief economist Philip Shaw said he did not expect the poor
retail figures to have any immediate effect on interest rates.


In [78]:
from transformers import pipeline

# Load a summarization pipeline
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


In [79]:
text = doc.iloc[0].split('\n', 1)[1]

In [80]:
# Generate summary
summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
print("Summary:")
print(summary[0]['summary_text'])

Summary:
 Retail sales dropped by 1% on the month in December, after a 0.6% rise in November . Clothing retailers and non-specialist stores were the worst hit with only internet retailers showing any significant growth .
